# 🏆 Competition Winner - Google Colab Edition

This notebook provides an interactive interface for running the enhanced gpt-oss-20b vulnerability competition in Google Colab.

## Features
- 60+ sophisticated attack scenarios
- Multiple testing strategies (standard, aggressive, subtle)
- Four scanning modes (comprehensive, targeted, random, quick)
- Automatic memory management for Colab GPUs
- Detailed result analysis and visualization

## 1. Setup Environment

First, let's clone the repository and set up the environment.

In [ ]:
# Clone the repository (if not already done)
import os
if not os.path.exists('/content/openai-'):
    !git clone https://github.com/Unknown1502/openai-.git /content/openai-
    %cd /content/openai-
else:
    %cd /content/openai-
    !git pull

In [ ]:
# Run the Colab-specific setup
!python COLAB_COMPETITION_WINNER.py

## 2. Memory Monitoring

Let's set up memory monitoring to track GPU usage during the competition.

In [ ]:
import torch
import psutil
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

def monitor_resources():
    """Monitor and display system resources."""
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.memory_allocated() / 1e9
        gpu_total = torch.cuda.get_device_properties(0).total_memory / 1e9
        gpu_percent = (gpu_memory / gpu_total) * 100
        
        print(f"🖥️ GPU: {torch.cuda.get_device_name(0)}")
        print(f"📊 GPU Memory: {gpu_memory:.2f}/{gpu_total:.2f} GB ({gpu_percent:.1f}%)")
    
    cpu_percent = psutil.cpu_percent()
    ram_percent = psutil.virtual_memory().percent
    
    print(f"💻 CPU Usage: {cpu_percent}%")
    print(f"🧠 RAM Usage: {ram_percent}%")

# Display current resources
monitor_resources()

## 3. Quick Test Run

Let's run a quick test to ensure everything is working correctly.

In [ ]:
# Test the enhanced attack vectors
from src.competition.enhanced_attack_vectors import (
    ENHANCED_COMPETITION_ATTACK_VECTORS,
    get_categories,
    get_attack_count
)

print(f"✅ Loaded {len(ENHANCED_COMPETITION_ATTACK_VECTORS)} attack scenarios")
print(f"\n📊 Attack Distribution by Category:")

attack_counts = get_attack_count()
for category, count in sorted(attack_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {category}: {count} attacks")

In [ ]:
# Visualize attack distribution
import matplotlib.pyplot as plt

categories = list(attack_counts.keys())
counts = list(attack_counts.values())

plt.figure(figsize=(12, 6))
plt.bar(categories, counts, color='skyblue', edgecolor='navy')
plt.xlabel('Attack Category')
plt.ylabel('Number of Scenarios')
plt.title('Distribution of Attack Scenarios by Category')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 4. Run Competition - Quick Mode

Let's start with a quick scan to test the setup.

In [ ]:
# Run quick competition scan
!python run_enhanced_competition.py --mode quick --strategy standard --output-dir quick_results

In [ ]:
# Load and display quick results
import json
import pandas as pd

try:
    with open('quick_results/enhanced_competition_report_standard_quick.json', 'r') as f:
        quick_report = json.load(f)
    
    print("🏆 Quick Scan Results:")
    print(f"Scenarios Tested: {quick_report['summary']['total_scenarios_tested']}")
    print(f"Vulnerabilities Found: {quick_report['summary']['vulnerabilities_found']}")
    print(f"Success Rate: {quick_report['summary']['success_rate']}")
    
    if quick_report['findings']:
        print("\n🔍 Sample Findings:")
        for i, finding in enumerate(quick_report['findings'][:3]):
            print(f"\n{i+1}. {finding['issue_summary']['issue_title']}")
            print(f"   Category: {finding['issue_summary']['topic_area']}")
            print(f"   Severity: {finding['metadata']['scenario_severity']}")
except FileNotFoundError:
    print("No results file found. Make sure the scan completed successfully.")

## 5. Run Full Competition Scans

Now let's run different competition strategies. Choose based on your available time and resources.

In [ ]:
# Strategy selector
from ipywidgets import interact, widgets

strategy_widget = widgets.Dropdown(
    options=['standard', 'aggressive', 'subtle'],
    value='standard',
    description='Strategy:',
)

mode_widget = widgets.Dropdown(
    options=['quick', 'random', 'targeted', 'comprehensive'],
    value='targeted',
    description='Mode:',
)

def run_competition(strategy, mode):
    print(f"Running competition with strategy='{strategy}' and mode='{mode}'...")
    output_dir = f"{strategy}_{mode}_results"
    !python run_enhanced_competition.py --strategy {strategy} --mode {mode} --output-dir {output_dir}

print("Select your competition parameters:")
display(strategy_widget)
display(mode_widget)

run_button = widgets.Button(description="Run Competition")
run_button.on_click(lambda b: run_competition(strategy_widget.value, mode_widget.value))
display(run_button)

## 6. Analyze Results

Let's analyze the competition results with visualizations.

In [ ]:
import glob
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load all report files
report_files = glob.glob('*_results/enhanced_competition_report_*.json')
all_reports = []

for file in report_files:
    with open(file, 'r') as f:
        report = json.load(f)
        report['file'] = file
        all_reports.append(report)

if all_reports:
    # Create comparison dataframe
    comparison_data = []
    for report in all_reports:
        comparison_data.append({
            'Strategy': report['summary']['strategy'],
            'Mode': report['summary']['mode'],
            'Scenarios': report['summary']['total_scenarios_tested'],
            'Vulnerabilities': report['summary']['vulnerabilities_found'],
            'Success Rate': float(report['summary']['success_rate'].rstrip('%'))
        })
    
    df = pd.DataFrame(comparison_data)
    
    # Display results table
    print("📊 Competition Results Summary:")
    display(df)
    
    # Visualize success rates
    plt.figure(figsize=(10, 6))
    sns.barplot(data=df, x='Strategy', y='Success Rate', hue='Mode')
    plt.title('Success Rate by Strategy and Mode')
    plt.ylabel('Success Rate (%)')
    plt.show()
else:
    print("No report files found. Run some competitions first!")

In [ ]:
# Analyze vulnerability categories across all reports
if all_reports:
    category_vulnerabilities = {}
    
    for report in all_reports:
        for finding in report['findings']:
            category = finding['issue_summary']['topic_area']
            if category not in category_vulnerabilities:
                category_vulnerabilities[category] = 0
            category_vulnerabilities[category] += 1
    
    # Create pie chart
    if category_vulnerabilities:
        plt.figure(figsize=(10, 8))
        plt.pie(category_vulnerabilities.values(), 
                labels=category_vulnerabilities.keys(), 
                autopct='%1.1f%%',
                startangle=90)
        plt.title('Vulnerability Distribution by Category')
        plt.axis('equal')
        plt.show()
        
        # Top recommendations
        print("\n🎯 Top Recommendations:")
        latest_report = all_reports[-1]
        for i, rec in enumerate(latest_report['recommendations'][:5], 1):
            print(f"{i}. {rec}")

## 7. Export Results

Export your findings for submission or further analysis.

In [ ]:
# Combine all findings
all_findings = []
for report in all_reports:
    all_findings.extend(report['findings'])

# Remove duplicates based on issue title
unique_findings = {}
for finding in all_findings:
    title = finding['issue_summary']['issue_title']
    if title not in unique_findings:
        unique_findings[title] = finding

# Save consolidated findings
consolidated_report = {
    'total_unique_findings': len(unique_findings),
    'findings': list(unique_findings.values()),
    'metadata': {
        'reports_analyzed': len(all_reports),
        'total_scenarios_tested': sum(r['summary']['total_scenarios_tested'] for r in all_reports),
        'timestamp': datetime.utcnow().isoformat()
    }
}

with open('consolidated_competition_results.json', 'w') as f:
    json.dump(consolidated_report, f, indent=2)

print(f"✅ Exported {len(unique_findings)} unique findings to consolidated_competition_results.json")

# Create submission-ready format
submission_findings = []
for finding in list(unique_findings.values())[:10]:  # Top 10 findings
    submission_findings.append({
        'title': finding['issue_summary']['issue_title'],
        'category': finding['issue_summary']['topic_area'],
        'severity': finding['metadata']['scenario_severity'],
        'description': finding['issue_summary']['overview']
    })

with open('submission_findings.json', 'w') as f:
    json.dump({'findings': submission_findings}, f, indent=2)

print("✅ Created submission_findings.json with top findings")

In [ ]:
# Download results
from google.colab import files

print("📥 Download your results:")
files.download('consolidated_competition_results.json')
files.download('submission_findings.json')

## 8. Cleanup

Clean up GPU memory and temporary files.

In [ ]:
# Clear GPU memory
import torch
import gc

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    gc.collect()
    print("✅ GPU memory cleared")

# Final resource check
monitor_resources()

## 🎉 Congratulations!

You've successfully run the enhanced competition scanner. Your findings are ready for submission.

### Next Steps:
1. Review the consolidated findings
2. Select the most impactful vulnerabilities for submission
3. Add any manual findings you discovered
4. Submit your results to the competition

Good luck! 🏆